In [7]:
import tensorflow as tf 
import numpy as np
from tensorflow.keras import layers
import tensorflow_datasets as tfds

Define the model and initialize it.

In [8]:
def base_model(): 
  input_ = layers.Input(shape=(784,), name="input")
  dense1 = layers.Dense(64, activation="relu", name="dense1")(input_)
  dense2 = layers.Dense(64, activation="relu", name="dense2")(dense1)
  output = layers.Dense(10, activation="softmax", name="output")(dense2)
  model = tf.keras.models.Model(inputs = input_, outputs= output)
  return model

model = base_model()

Build the data pipeline using TFDS Fashion MNIST

In [9]:
train_data = tfds.load("fashion_mnist", split="train")
test_data = tfds.load("fashion_mnist", split="test")

def preprocess_func(data):
  #each item in the Tensorflow Dataset object has an 'image' and 'label'. 
  image = data["image"]
  #flatten 
  image = tf.reshape(image, [-1]) #this syntax tells, I want just one dimension, you infer the size of it based on input.
  #cast to float and normalize.
  image = tf.cast(image, dtype=tf.float32)
  image = image/255. #squeeze pixel values between 0 and 1.
  return image, data["label"]


train = train_data.map(preprocess_func)
test = test_data.map(preprocess_func)

BATCH_SIZE = 64

train_prefetch = train.shuffle(buffer_size=1024).batch(BATCH_SIZE)
test_prefetch = test.batch(BATCH_SIZE) #no need to shuffle the test set.


Model Configs

In [10]:
criterion = tf.keras.losses.SparseCategoricalCrossentropy() 
optimizer = tf.keras.optimizers.Adam() 


train_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy() 

In [11]:
def apply_gradient(optimizer, model, xs, ys, lr): 
  with tf.GradientTape() as tape: 
    logits = model(xs)
    loss = criterion(ys, logits)
    gradients = tape.gradient(loss, model.trainable_weights)
  
  optimizer.apply_gradients(zip(gradients, model.trainable_weights)) # in this case, this is going to be an adam object.
  return logits, loss


def train_single_epoch():
  losses = [] #accumulates losses from each batch in this epoch.
  #for each batch
  for step, (image_batch, label_batch) in enumerate(train_prefetch):
    logits, loss_value = apply_gradient(optimizer, model, image_batch, label_batch, 0.001) #hardcoding the lr here... 
    losses.append(loss_value)
    train_acc_metric.update_state(label_batch, logits) # accumulate the metric in each batch.

  return losses 

def validation_single_epoch():
  losses = [] 
  for step, (image_batch, label_batch) in enumerate(test_prefetch):
    val_logits = model(image_batch)
    val_loss = criterion(label_batch, val_logits)
    losses.append(val_loss)
    val_acc_metric.update_state(label_batch, val_logits) # accumulating the metric.

  return losses



Main training loop.

In [12]:
EPOCHS = 20 

tr_metrics = [] 
val_metrics = [] 

epoch_tr_losses = [] 
epoch_val_losses = [] 

for epoch in range(EPOCHS):
   epoch_loss = np.mean(train_single_epoch())
   epoch_tr_losses.append(epoch_loss)
   
   epoch_acc = train_acc_metric.result()
   tr_metrics.append(epoch_acc)
   train_acc_metric.reset_states()
  
   epoch_val_loss = np.mean(validation_single_epoch())
   epoch_val_losses.append(epoch_val_loss)

   val_acc = val_acc_metric.result()
   val_metrics.append(val_acc)
   val_acc_metric.reset_states()

   print(f"Epoch: {epoch}, loss: {epoch_loss}, acc: {epoch_acc}, val_loss: {epoch_val_loss}, val_acc: {val_acc}")



   


Epoch: 0, loss: 0.5411069989204407, acc: 0.8104166388511658, val_loss: 0.44139793515205383, val_acc: 0.8429999947547913
Epoch: 1, loss: 0.3934357166290283, acc: 0.8582833409309387, val_loss: 0.4103740155696869, val_acc: 0.8514999747276306
Epoch: 2, loss: 0.3523063361644745, acc: 0.8726833462715149, val_loss: 0.4024989604949951, val_acc: 0.8585000038146973
Epoch: 3, loss: 0.3274795711040497, acc: 0.879883348941803, val_loss: 0.36439579725265503, val_acc: 0.8709999918937683
Epoch: 4, loss: 0.31343191862106323, acc: 0.8851666450500488, val_loss: 0.37132149934768677, val_acc: 0.868399977684021
Epoch: 5, loss: 0.2950303554534912, acc: 0.8914666771888733, val_loss: 0.3520917296409607, val_acc: 0.8770999908447266
Epoch: 6, loss: 0.287485271692276, acc: 0.8942000269889832, val_loss: 0.3667902946472168, val_acc: 0.8705000281333923
Epoch: 7, loss: 0.27487289905548096, acc: 0.8976666927337646, val_loss: 0.35601145029067993, val_acc: 0.8755000233650208
Epoch: 8, loss: 0.2655051350593567, acc: 0.90

In [13]:
t = train_single_epoch()